In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## GANS

- [introduction to GANs](https://wiki.pathmind.com/generative-adversarial-network-gan) 
- [deep conv GANs paper](https://arxiv.org/abs/1511.06434)
- [difusion vs. GANs]("https://arxiv.org/abs/2105.05233")

In [2]:
import os
import torch
import torchvision
from torch.utils.data import DataLoader , Dataset
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image


In [3]:
print(f" monet: {len(os.listdir('/kaggle/input/gan-getting-started/monet_jpg'))}", 
     f"photos: {len(os.listdir('/kaggle/input/gan-getting-started/photo_jpg'))}")

 monet: 300 photos: 7038


In [4]:
# Discriminator network
class Discriminator(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
        nn.Conv2d(in_channels= in_channel, out_channels= out_channel,
                  kernel_size= (5,5), stride= (2,2), padding= (1,1)), 
            nn.LeakyReLU(0.2),
            self.block(out_channel, 2*out_channel, 5, 2, 1),
            self.block(2*out_channel, 4*out_channel, 5,2,1),
            self.block(4*out_channel, 8*out_channel, 5,2,1),
            nn.Conv2d(8*out_channel, 1 ,5, 2, 0 ),
            nn.Sigmoid()
            
        )
    
    def block( self, in_channel, out_channel, kernel_size, stride, padding):
        return nn.Sequential(
        nn.Conv2d(in_channels= in_channel , out_channels = out_channel,
                 kernel_size= kernel_size ,stride= stride 
                  , padding = padding, bias = False),
            nn.BatchNorm2d(out_channel),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        return self.disc(x)
        
        

In [5]:
class Generator(nn.Module):
    def __init__(self, in_channel, out_channel , output_channel):
        
        super(Generator , self).__init__()
        
        self.Gen = nn.Sequential( self.block(in_channel, out_channel*16, 5,2,1),
                                self.block(out_channel*16, out_channel*8, 5,2,1),
                                self.block(in_channel*8, out_channel*4, 5,2,1),
                                self.block(in_channel*4, out_channel*2, 5,2,1),
                                nn.ConvTranspose2d(out_channel*2 , output_channel, 5,2,1),
                                nn.ReLU(),
                                nn.Tanh()
                                    )
        
        
        
    def block(self, in_channel, out_channel, kernel_size, stride, padding):
        return nn.Sequential(nn.ConvTranspose2d(in_channel, out_channel,
                                                kernel_size , stride, padding), 
                             nn.BatchNorm2d(out_channel),
                            nn.ReLU())
    
    def forward(self, x):
        return self.Gen(x)